In [34]:
from torchvision.models import resnet18
from torchvision.datasets import EMNIST
from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader
import torch

In [35]:
model = resnet18(num_classes=10) # MNIST has 10 classes
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [36]:
transforms = transforms.Compose([
                                 transforms.ToTensor()
])

In [37]:
train_ds = EMNIST('EMNIST', split='digits', train=True, transform=transforms, download=False)
test_ds = EMNIST('EMNIST', split='digits', train=False, transform=transforms, download=False)
train_dl = DataLoader(train_ds, batch_size=1024, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1024)

In [38]:
class ResNetMNIST(nn.Module):
    def __init__(self):
        super().__init__()
        # define model and loss
        self.model = resnet18(num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # this decorator automatically handles moving your tensors to GPU if required
    def forward(self, x):
        return self.model(x)
  
    def training_step(self, batch, batch_no):
        # implement single training step
        x, y = batch
        logits = self(x)
        return logits


In [40]:
!ls

data		     project-Mahmoud.ipynb
EMNIST		     project_new.ipynb
EMNIST.ipynb	     project-segmentation.ipynb
lightning_logs	     project-segmentation-prediction-Copy1.ipynb
media		     project-segmentation-prediction.ipynb
MNIST.ipynb	     __pycache__
predictions.ipynb    README.md
project-Copy1.ipynb  saved_model.pth
project-final.ipynb  train_games
project.ipynb	     utils.py


In [43]:
model = ResNetMNIST()
model.load_state_dict(torch.load('saved_model.pth', map_location=torch.device('cpu')))
model.eval()

ResNetMNIST(
  (model): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [44]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dl:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 40000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 40000 test images: 99 %


In [45]:
correct / total

0.9928

In [46]:
predicted

tensor([6, 1, 9, 5, 7, 4, 0, 3, 6, 6, 5, 5, 3, 1, 2, 7, 5, 5, 5, 1, 7, 0, 5, 8,
        2, 7, 5, 8, 4, 3, 6, 4, 9, 2, 1, 7, 8, 3, 0, 2, 0, 6, 3, 0, 7, 8, 8, 2,
        3, 4, 2, 0, 2, 5, 6, 6, 3, 5, 0, 9, 7, 3, 7, 4])

In [31]:
model = ResNetMNIST()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.005)

In [32]:
import numpy as np
from tqdm import tqdm

epochs = 10
min_valid_loss = np.inf

for e in tqdm(range(epochs)):
    train_loss = 0.0
    model.train()     # Optional when not using Model Specific layer
    for data, labels in train_dl:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        target = model(data)
        loss = criterion(target,labels)
        loss.backward()
        optimizer.step()
        train_loss = loss.item() * data.size(0)
    
    valid_loss = 0.0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in test_dl:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        target = model(data)
        loss = criterion(target,labels)
        valid_loss = loss.item() * data.size(0)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

  0%|          | 0/10 [01:43<?, ?it/s]


KeyboardInterrupt: 